In [1]:
import sys
sys.path.append('/Users/spindicate/Documents/programming/zooscraper/')

In [2]:
import django
django.setup()

In [3]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
from datetime import datetime as dt
from scraper.models import MonthlySales

In [5]:
import pdfplumber

In [6]:
root_path = '/Users/spindicate/Documents/docs/covid19/treb/'

In [7]:
path = root_path + 'mw2002.pdf'
pdf = pdfplumber.open(path)
page = pdf.pages[-1]
tables = page.find_tables()
_2019 = tables[1].extract()
_2020 = tables[2].extract()

In [8]:
path = root_path + 'mw1812.pdf'
pdf = pdfplumber.open(path)
page = pdf.pages[-1]
tables = page.find_tables()
_2017 = tables[1].extract()
_2018 = tables[2].extract()

In [9]:
path = root_path + 'mw1612.pdf'
pdf = pdfplumber.open(path)
page = pdf.pages[-1]
tables = page.find_tables()
_2015 = tables[1].extract()
_2016 = tables[2].extract()

In [10]:
tables = {2020: _2020, 2019: _2019, 2018: _2018, 2017: _2017, 2016: _2016, 2015: _2015}

In [11]:
monthly_sales = []
for year, month in tables.items():
    for values in month:
        if values[1] == '-' or values[0] in ['Year to Date', 'Annual']:
            pass
        else:
            monthly_sales.append(
                MonthlySales( 
                    month=dt.strptime(values[0], '%B').month,
                    year=year,
                    number=float(values[1].replace(',', '')), 
                    avg_price=float(values[2].replace('$', '').replace(',', ''))
                )
            )

In [56]:
# MonthlySales.objects.bulk_create(monthly_sales)

In [29]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Range1d, LinearAxis, HoverTool

In [50]:
output_notebook()

Loading BokehJS ...

In [52]:
def monthly_chart(month):
    month_str = dt(1900, month, 1).strftime('%B')
    years = list(reversed(list(str(yr) for yr in tables.keys())))
    counts = list(reversed(MonthlySales.objects.filter(month=month).values_list('number', flat=True)))
    growths = [count / counts[i - 1] - 1 for i, count in enumerate(counts) if i > 0]
    growths.insert(0, 0)

    p = figure(x_range=years, plot_height=250, title=month_str,
               toolbar_location=None, tools="")

    p.vbar(x=years, top=counts, width=0.9)

    p.extra_y_ranges['growth'] = Range1d(start=-.5, end=.5)

    p.add_layout(LinearAxis(y_range_name='growth', axis_label="Growth",), 'right')
    p.line(x=years, y=growths, color="red", line_width=2, y_range_name='growth')

    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    
    return p

In [53]:
ps= [monthly_chart(month) for month in [1,2,3]]

for p in ps:
    show(p)